# ORCL Stock Article extractions

In [3]:
import requests

# api_key = '23f5208b7e2940aeb8b72c67195565b2'
api_key = 'bd9bb272078b41ebb0ee0a905d4a9532'
date_from = '2023-12-11'
date_to = '2024-01-08'
# Expanded list of sources
sources = 'bloomberg,cnbc,reuters,financial-times,techcrunch,the-wall-street-journal,the-verge,business-insider,the-economist,wired,engadget,bbc-news,fortune,techradar'

# Broadening the search query
query = 'Oracle OR "ORCL" OR "Larry Ellison" OR "Safra Catz" OR "Enterprise software" OR "Cloud services" OR "Database technology" OR "Oracle financials" OR "Technology"'

url = f'https://newsapi.org/v2/everything?q={query}&from={date_from}&to={date_to}&sources={sources}&apiKey={api_key}'
response = requests.get(url)
data = response.json()

# Check for 'articles' in response and extract headlines with publication dates
if 'articles' in data:
    news_data = [(article['title'], article['publishedAt'][:10]) for article in data['articles']]
    for headline, date in news_data:
        print(f"Date: {date}, Headline: {headline}")
else:
    print("Error or no articles found. Response:")
    print(data)


Date: 2023-12-23, Headline: Salesforce's year of the boomerang: Here are the C-suite executives who joined, left, or returned in 2023
Date: 2023-12-29, Headline: WIRED’s 2023 Year-in-Review Quiz
Date: 2023-12-14, Headline: My Surprisingly Unbiased Week With Elon Musk's 'Politically Biased' Chatbot
Date: 2023-12-19, Headline: Sam Ryder surprises shoppers in Reading
Date: 2023-12-12, Headline: Netflix is down for many people right now
Date: 2023-12-31, Headline: Google Fixes Nearly 100 Android Security Issues
Date: 2023-12-27, Headline: The top 15 people in enterprise artificial intelligence
Date: 2023-12-31, Headline: LG’s newest 32-inch 4K monitor is a looker with a smart TV inside
Date: 2024-01-08, Headline: The Creative’s Toolbox Gets an AI Upgrade
Date: 2024-01-05, Headline: Why people are flocking to these 10 'blue island' cities
Date: 2023-12-13, Headline: OpenAI will pay to train its models on Business Insider and Politico articles
Date: 2023-12-23, Headline: The Race to Put Brai

# Sentiment Analysis on News Articles

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('yiyanghkust/finbert-tone')
model = AutoModelForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone')

# Extract just the headlines from news_data
headlines = [headline for headline, _ in news_data]

# Tokenize and encode headlines
max_length = 512  # You can adjust this based on your requirements
inputs = tokenizer(headlines, padding=True, truncation=True, max_length=max_length, return_tensors="pt")

# Predict sentiment
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

# Pair each sentiment score with its corresponding date
sentiment_data = []
for (headline, date), prediction in zip(news_data, predictions):
    sentiment_data.append({
        'date': date,
        'headline': headline,
        'sentiment_positive': prediction[0].item(),
        'sentiment_neutral': prediction[1].item(),
        'sentiment_negative': prediction[2].item()
    })


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [5]:
sentiment_data

[{'date': '2023-12-23',
  'headline': "Salesforce's year of the boomerang: Here are the C-suite executives who joined, left, or returned in 2023",
  'sentiment_positive': 0.999996542930603,
  'sentiment_neutral': 1.1405345645698617e-07,
  'sentiment_negative': 3.3659684959275182e-06},
 {'date': '2023-12-29',
  'headline': 'WIRED’s 2023 Year-in-Review Quiz',
  'sentiment_positive': 0.9999856948852539,
  'sentiment_neutral': 3.3034064017556375e-07,
  'sentiment_negative': 1.3977928574604448e-05},
 {'date': '2023-12-14',
  'headline': "My Surprisingly Unbiased Week With Elon Musk's 'Politically Biased' Chatbot",
  'sentiment_positive': 0.999392032623291,
  'sentiment_neutral': 4.367200290289475e-06,
  'sentiment_negative': 0.0006035905098542571},
 {'date': '2023-12-19',
  'headline': 'Sam Ryder surprises shoppers in Reading',
  'sentiment_positive': 0.9820265173912048,
  'sentiment_neutral': 0.0007641530246473849,
  'sentiment_negative': 0.017209265381097794},
 {'date': '2023-12-12',
  'h

# Historical Stock Data and Sentiment Analysis Combined

In [9]:
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import StandardScaler

# Convert sentiment_data to a DataFrame
sentiment_df = pd.DataFrame(sentiment_data)

# Convert date to datetime and set as index
sentiment_df['date'] = pd.to_datetime(sentiment_df['date'])
sentiment_df.set_index('date', inplace=True)

# Aggregate sentiment scores by date
average_sentiment = sentiment_df.groupby('date').mean()

# Download stock data
ticker_symbol = 'ORCL'
stock_data = yf.download(ticker_symbol, start='2023-12-09', end='2024-01-08')

# Selecting the required columns
stock_data = stock_data[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]

# Convert stock_data index to datetime
stock_data.index = pd.to_datetime(stock_data.index)

# Combine stock data with sentiment data
combined_data = stock_data.join(average_sentiment)

# Interpolate missing values
combined_data = combined_data.interpolate(method='time')

# Scale price-related columns
price_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close']
scaler_prices = StandardScaler()
combined_data[price_cols] = scaler_prices.fit_transform(combined_data[price_cols])

# Scale Volume column independently
scaler_volume = StandardScaler()
combined_data['Volume'] = scaler_volume.fit_transform(combined_data[['Volume']])

print(combined_data)

[*********************100%%**********************]  1 of 1 completed

<ipython-input-9-b778d4ca54f7>:13: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.




                Open      High       Low     Close  Adj Close    Volume  \
Date                                                                      
2023-12-11  3.100199  3.514808  3.116712  3.327899   3.327899  0.381127   
2023-12-12 -0.633758 -0.266641 -0.902933 -1.267373  -1.267373  3.304181   
2023-12-13 -1.180189 -0.939303 -1.347832 -0.567813  -0.567813  1.136229   
2023-12-14 -0.913718 -1.248363 -1.158050 -1.427822  -1.427822  0.635767   
2023-12-15 -1.649042 -0.615698 -1.316721 -0.461916  -0.461916  1.198030   
2023-12-18 -0.394270  0.173318 -0.212251  0.077194   0.077194 -0.084379   
2023-12-19  0.135298  0.238763  0.375762  0.478318   0.478318 -0.342362   
2023-12-20  0.462481  0.147863  0.142424 -0.195569  -0.195569 -0.408802   
2023-12-21  0.040850  0.089689  0.288647  0.353167   0.353167 -0.491151   
2023-12-22  0.378155  0.293304  0.581099  0.462272   0.462272 -0.682051   
2023-12-26  0.654744  0.351482  0.543764  0.459065   0.459065 -0.675219   
2023-12-27  0.587284  0.

In [10]:
import pandas as pd
df = pd.DataFrame(combined_data)
df

,Open,High,Low,Close,Adj Close,Volume,sentiment_positive,sentiment_neutral,sentiment_negative
Date,,,,,,,,,
2023-12-11,3.100199,3.514808,3.116712,3.327899,3.327899,0.381127,0.272845,0.000014,0.727142
2023-12-12,-0.633758,-0.266641,-0.902933,-1.267373,-1.267373,3.304181,0.684577,0.000763,0.314660
2023-12-13,-1.180189,-0.939303,-1.347832,-0.567813,-0.567813,1.136229,0.999994,0.000001,0.000005
2023-12-14,-0.913718,-1.248363,-1.158050,-1.427822,-1.427822,0.635767,0.799861,0.200014,0.000124
2023-12-15,-1.649042,-0.615698,-1.316721,-0.461916,-0.461916,1.198030,0.999930,0.000004,0.000066
2023-12-18,-0.394270,0.173318,-0.212251,0.077194,0.077194,-0.084379,0.681974,0.002501,0.315525
2023-12-19,0.135298,0.238763,0.375762,0.478318,0.478318,-0.342362,0.995479,0.000195,0.004326
2023-12-20,0.462481,0.147863,0.142424,-0.195569,-0.195569,-0.408802,0.610149,0.260589,0.129261
2023-12-21,0.040850,0.089689,0.288647,0.353167,0.353167,-0.491151,0.731131,0.001684,0.267184


# Transformer Architecture

In [45]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
from tensorflow.keras.callbacks import EarlyStopping

selected_features = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'sentiment_positive', 'sentiment_neutral', 'sentiment_negative']
data = combined_data[selected_features]

train_size = int(len(data) * 0.7)
val_size = int(len(data) * 0.15)
train_data = data.iloc[:train_size]
val_data = data.iloc[train_size:train_size + val_size]
test_data = data.iloc[train_size + val_size:]

# Prepare features and target for model training
X_train = train_data.drop('Close', axis=1)
y_train = train_data['Close']
X_val = val_data.drop('Close', axis=1)
y_val = val_data['Close']
X_test = test_data.drop('Close', axis=1)
y_test = test_data['Close']

# Define and compile the neural network model
model = Sequential()
model.add(Dense(32, activation='LeakyReLU', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(16, activation='LeakyReLU'))
model.add(Dense(1))
model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(
    X_train, y_train,
    epochs=50,  # Set back to 50 or an arbitrarily large number
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping]  # Add the early stopping callback here
)

train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(train_loss) + 1)

# Convert the epochs range object to a list
epochs_list = list(epochs)

# Plotting the training and validation loss
loss_fig = go.Figure()
loss_fig.add_trace(go.Scatter(x=epochs_list, y=train_loss, mode='lines', name='Training Loss'))
loss_fig.add_trace(go.Scatter(x=epochs_list, y=val_loss, mode='lines', name='Validation Loss'))
loss_fig.update_layout(title='Training and Validation Loss per Epoch',
                       xaxis_title='Epochs',
                       yaxis_title='Loss',
                       legend_title='Type')
loss_fig.show()

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
print(f'MSE: {mse}, MAE: {mae}, R^2: {r2}')

# Reshape y_test and predictions for inverse transform
y_test_reshaped = y_test.values.reshape(-1, 1)
predictions_reshaped = predictions.reshape(-1, 1)

num_scaled_cols = 5

# Create separate dummy arrays for inverse scaling
dummy_array_y_test = np.zeros((len(y_test_reshaped), num_scaled_cols))
dummy_array_predictions = np.zeros((len(predictions_reshaped), num_scaled_cols))

# Fill in the 'Close' column values in the dummy arrays
# Assuming 'Close' is the last of the scaled columns
dummy_array_y_test[:, -1] = y_test_reshaped.flatten()
dummy_array_predictions[:, -1] = predictions_reshaped.flatten()

# Inverse transform the 'Close' prices using the dummy arrays
y_test_original = scaler_prices.inverse_transform(dummy_array_y_test)[:, -1]
predictions_original = scaler_prices.inverse_transform(dummy_array_predictions)[:, -1]

# Extracting testing dates
test_dates = test_data.index

# Plotting with Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=test_dates, y=y_test_original, mode='lines', name='Actual'))
fig.add_trace(go.Scatter(x=test_dates, y=predictions_original, mode='lines', name='Predicted'))
fig.update_layout(title='Actual vs Predicted Stock Prices', xaxis_title='Date', yaxis_title='Stock Price', legend_title='Legend')
fig.show()

Epoch 1/50
1/1 [==============================] - 1s 987ms/step - loss: 0.5437 - val_loss: 0.0353
Epoch 2/50
1/1 [==============================] - 0s 52ms/step - loss: 0.1655 - val_loss: 0.1101
Epoch 3/50
1/1 [==============================] - 0s 43ms/step - loss: 0.2530 - val_loss: 0.2278
Epoch 4/50
1/1 [==============================] - 0s 70ms/step - loss: 0.2180 - val_loss: 0.2796
Epoch 5/50
1/1 [==============================] - 0s 64ms/step - loss: 0.3940 - val_loss: 0.2261
Epoch 6/50
1/1 [==============================] - 0s 55ms/step - loss: 0.1204 - val_loss: 0.1447
Epoch 6: early stopping


1/1 [==============================] - 0s 72ms/step
MSE: 0.004567613943268067, MAE: 0.06487746476062276, R^2: 0.8924568741803391


In [46]:
model.summary()

Model: "sequential_35"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_105 (Dense)           (None, 32)                288       
                                                                 
 dropout_35 (Dropout)        (None, 32)                0         
                                                                 
 dense_106 (Dense)           (None, 16)                528       
                                                                 
 dense_107 (Dense)           (None, 1)                 17        
                                                                 
Total params: 833 (3.25 KB)
Trainable params: 833 (3.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
